In [1]:
import cv2
import os
from PIL import Image
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.utils import normalize
from keras.models import Sequential
from keras.layers import MaxPooling2D, Conv2D, Dropout, Activation, Flatten, Dense
from keras.utils import to_categorical




2024-10-28 07:33:16.245314: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-28 07:33:16.250226: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-28 07:33:16.265812: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-28 07:33:16.290148: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-28 07:33:16.297152: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-28 07:33:16.317185: I tensorflow/core/platform/cpu_feature_gu

In [2]:

# image directory
image_dir = 'datasets/'

# no tumor list

dataset = []
labels = []
INPUT_SIZE = 64

In [3]:
# no tumor image list
no_tumor = os.listdir(image_dir + 'no/')
for i, image_name in enumerate(no_tumor):
    if image_name.split('.')[1] == 'jpg':
        # read the images using open cv
        image = cv2.imread(image_dir + 'no/' + image_name)

        # convert it to rgb using pillow
        image = Image.fromarray(image, 'RGB')

        # resize the image
        image = image.resize((INPUT_SIZE, INPUT_SIZE))
        # set the datasets as numpy arrays

        dataset.append(np.array(image))

        labels.append(0)

In [4]:

# yes tumor list
yes_tumor = os.listdir(image_dir + 'yes/')

for i, image_name in enumerate(yes_tumor):
    if image_name.split('.')[1] == 'jpg':
        # read the images using open cv
        image = cv2.imread(image_dir + 'yes/' + image_name)

        # convert it to rgb using pillow
        image = Image.fromarray(image, 'RGB')

        # resize the image
        image = image.resize((INPUT_SIZE, INPUT_SIZE))
        # set the datasets as numpy arrays

        dataset.append(np.array(image))

        labels.append(1)
        

In [5]:

# convert the datasets and labels into a numpy array

dataset = np.array(dataset)
labels = np.array(labels)


In [6]:

# 20% data for testing
x_train, x_test, y_train, y_test = train_test_split(dataset, labels, test_size=0.2)

# reshape (no_of_images, image_width,image_height,no_of_channels)

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)


(2400, 64, 64, 3)
(2400,)
(600, 64, 64, 3)
(600,)


In [7]:
x_train = normalize(x_train, axis=1)
x_test = normalize(x_test, axis=1)
y_train = to_categorical(y_train,num_classes=2)
y_test = to_categorical(y_test,num_classes=2)

In [8]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(INPUT_SIZE, INPUT_SIZE, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

/home/kupamakunura/miniconda3/envs/brain-tumor/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [9]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', kernel_initializer='he_uniform'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))


In [10]:
model = Sequential()
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', kernel_initializer='he_uniform'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))



In [11]:
model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(2))
model.add(Activation('softmax'))

In [12]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [13]:
model.fit(x_train, y_train, epochs=10, batch_size=16, verbose=1, validation_data=(x_test, y_test), shuffle=False)
model.save('sequential-model.h5')

Epoch 1/10


2024-10-28 07:36:53.713912: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 117964800 exceeds 10% of free system memory.


150/150 ━━━━━━━━━━━━━━━━━━━━ 24s 147ms/step - accuracy: 0.6922 - loss: 0.7265 - val_accuracy: 0.8400 - val_loss: 0.3760
Epoch 2/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 50s 209ms/step - accuracy: 0.8153 - loss: 0.4004 - val_accuracy: 0.8817 - val_loss: 0.3255
Epoch 3/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 32s 213ms/step - accuracy: 0.8880 - loss: 0.2800 - val_accuracy: 0.9250 - val_loss: 0.2011
Epoch 4/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 24s 100ms/step - accuracy: 0.9229 - loss: 0.1995 - val_accuracy: 0.9500 - val_loss: 0.1589
Epoch 5/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 28s 153ms/step - accuracy: 0.9445 - loss: 0.1476 - val_accuracy: 0.9717 - val_loss: 0.1232
Epoch 6/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 47s 193ms/step - accuracy: 0.9416 - loss: 0.1177 - val_accuracy: 0.9717 - val_loss: 0.1206
Epoch 7/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 21s 142ms/step - accuracy: 0.9577 - loss: 0.1061 - val_accuracy: 0.9700 - val_loss: 0.1293
Epoch 8/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 49s 197ms/step - accuracy: 0.9613 - loss: 0.0889 - val